In [2]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [67 lines of output]
      /home/jongsong/anaconda3/envs/conv_ai/lib/python3.12/site-packages/setuptools/__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
      !!
      
              ********************************************************************************
              Requirements should be satisfied by a PEP 517 installer.
              If you are using pip, you can try `pip install --use-pep517`.
              ********************************************************************************
      
      !!
        dist.fetch_build_eggs(dist.setup_requires)
        error: subprocess-exited-with-error
      
        × python setup.py egg_info did not run successfully.
        │ exit code: 1
        ╰─> [1 lines of output]
            ERROR: Can not execute `setup.py` since

In [9]:
import calendar_utils
import json

In [13]:
import importlib 
importlib.reload(calendar_utils)

<module 'calendar_utils' from '/home/jongsong/conv_ai/calendar_utils.py'>

In [3]:
from dotenv import load_dotenv
import os
import re
import datetime

load_dotenv()

True

In [4]:
# Basic ChatGPT class
import openai
from openai import OpenAI


class BasicChatGPT:
    def __init__(self, openai_api_key, max_tokens=1000):
        openai.api_key = openai_api_key
        self.max_tokens = max_tokens
        self.messages = []
        self.client = OpenAI()

    def call(self):
        response = self.client.chat.completions.create(
          model="gpt-4-1106-preview",
          # model="gpt-3.5-turbo",
          response_format={ "type": "json_object" },
          messages=self.messages,
          max_tokens=self.max_tokens
        )
        return response

    def prompt(self, text):
        # Update the messages so far
        self.messages.append({
            "role": "user",
            "content": text,
        })

        # Call ChatGPT
        response = self.call()

        # Save the returned message
        message = response["choices"][0]["message"]
        self.messages.append(message)

        return message["content"]


In [6]:
# Make a ChatGPT object
basic_chatgpt = BasicChatGPT(os.environ.get("OPENAI_API_KEY"), max_tokens = 120)

# run_console(basic_chatgpt)

In [10]:
# calendar service 정의하는 부분 -> credentials.json 이 경로에 있어야 하고 
# 그리고 팝업 뜨면 너가 authorize 해줘야함 그리고 너 노트북이나 데탑 로컬에서 해야 함
# https://developers.google.com/resources/api-libraries/documentation/calendar/v3/python/latest/calendar_v3.events.html#list
service = calendar_utils.get_calendar_service()

In [147]:
## if user asks about "today's schedule"
## first detect today
## then find what's today
## then request calendar for today's schedule

In [33]:
# Example Query
user_input = "I want to see a list of schedule from Monday two weeks later."

In [58]:
# first detect today and then find what date is today

input_text = f""" Today is {datetime.datetime.today().date()}.
Detect any time-related phrase from the given input from the user and resolve it into a date in the format of YYYY/MM/DD and the date after that day YYYY/MM/DD+1day.
Examples include today, tomorrow, this Wednesday, next Tuesday, last Friday, 11/13, November 5th, 13th of July.
When the detected word is day of week, make sure to include adjective in front of it such as "this", "next", "upcoming", "last", "past".
\n\n
Your output must be in this format. Json("detected_phrase": <detected phrase>, "date": <YYYY/MM/DD>, "date_after_date": <YYYY/MM/DD>)

\n\n
Example 1
Input: Today is 2023/1/2. ... I want to schedule a meeting at 5PM tomorrow.
Output: tomorrow, 2023/1/3, 2023/1/4

Example 2
Input: Today is 2023/1/2. ... What time does the class start next Friday?
Output: next Friday, 2023/1/13, 2023/1/14

Example 3
Input: Today is 2023/1/2. ... What is the date of this upcoming Friday?
Output: this upcoming Friday, 2023/1/6, 2023/1/7
\n\nInput: 
"""

input_text += user_input



basic_chatgpt.messages.append({
    "role": "user",
    "content": input_text,
})

# Call ChatGPT
response = basic_chatgpt.call()
message = response.choices[0].message.content
init_result = json.loads(message)
date_expression = init_result['detected_phrase'] # detected_phrase
date_min = init_result['date'] #date
date_max = init_result['date_after_date'] #date+1
# Save the returned message
# message = response["choices"][0]["message"]
# self.messages.append(message)



In [35]:
# mydate = datetime.datetime.strptime(init_result['date'], "%Y/%m/%d").isoformat() + "Z"
# mydate_p1 = (datetime.datetime.strptime(init_result['date'], "%Y/%m/%d") + datetime.timedelta(days=1)).isoformat() + "Z"
mydate = datetime.datetime.now(datetime.UTC).isoformat().split('+')[0] + "Z"
mydate_p1 = (datetime.datetime.now(datetime.UTC) + datetime.timedelta(days=1)).isoformat().split('+')[0] + "Z"
print(mydate, mydate_p1)

2023-11-21T13:19:31.083303Z 2023-11-22T13:19:31.083401Z


In [50]:
#https://developers.google.com/resources/api-libraries/documentation/calendar/v3/python/latest/calendar_v3.events.html#list
import dateutil.parser as parser
date_min = parser.parse(date_min)
date_min = date_min.isoformat().split('+')[0] + "Z"
date_max = parser.parse(date_max)
date_max = date_max.isoformat().split('+')[0] + "Z"
print(date_min)
event_list = calendar_utils.get_event_list_recent(timeMin=date_min, timeMax=date_max)
# print(event_list)
event_list_new = []
keys_to_extract = ['summary', 'start', 'organizer', 'end', 'location', 'attendees', ]
for event in event_list:
    event_dict = {}
    for k in keys_to_extract:
        try:
            if k == 'start' or k == 'end':
                event_dict[k] = event[k]
                print(event_dict[k])
                event_dict[k]['dateTime'] = event_dict[k]['dateTime'].replace(event_dict[k]['dateTime'][:10], date_min)
            else:
                event_dict[k] = event[k]
        except KeyError:
            event_dict[k] = ''
    event_list_new.append(event_dict)


2023-12-04T00:00:00Z
2023-09-04T15:30:00+09:00 데사프
2023-09-04T14:10:00+09:00 종현-재윤
2023-09-18T14:10:00+09:00 종현-재윤
2023-10-16T19:00:00+09:00 [P4DS] SSSCI Meeting
2023-12-04T12:15:00+09:00 기숙사 입주신청
{'dateTime': '2023-09-04T15:30:00+09:00', 'timeZone': 'Asia/Seoul'}
{'dateTime': '2023-09-04T17:00:00+09:00', 'timeZone': 'Asia/Seoul'}
{'dateTime': '2023-09-04T14:10:00+09:00', 'timeZone': 'Asia/Seoul'}
{'dateTime': '2023-09-04T14:40:00+09:00', 'timeZone': 'Asia/Seoul'}
{'dateTime': '2023-09-18T14:10:00+09:00', 'timeZone': 'Asia/Seoul'}
{'dateTime': '2023-09-18T14:40:00+09:00', 'timeZone': 'Asia/Seoul'}
{'dateTime': '2023-10-16T19:00:00+09:00', 'timeZone': 'Asia/Seoul'}
{'dateTime': '2023-10-16T20:00:00+09:00', 'timeZone': 'Asia/Seoul'}
{'dateTime': '2023-12-04T12:15:00+09:00', 'timeZone': 'Asia/Seoul'}
{'dateTime': '2023-12-04T12:45:00+09:00', 'timeZone': 'Asia/Seoul'}


In [87]:
# first detect today and then find what date is today
calendar_chatgpt = BasicChatGPT(os.environ.get("OPENAI_API_KEY"), max_tokens = 1000)
input_text = f''' You are a sophisticated calendar management assistant, adapt at organizing and managing calendar schedules for both simple and complex tasks.\

For a given day, check the user's Calendar input which is given as list of python dictionaries and output the agenda for the day in markdown using relevant emojis as bullet points. \
Your output must be in this format. Json("date": <YYYY/MM/DD>, "schedule": <schedule>, "start_time":<HH:MM>, "Location": <location>, "Participants":<participants>)
Here's an example:\

Example 1
Input: The given date is 2023-11-21. Which schedule do I have on the given day?\
Output: Schedule is Check-in at HyattRegency Seattle, Start time ⏰ is After 4:00 PM, Location is Hyatt Regency, Seattle, Participants are\
Sheryl Soo(sheryl@zapier.com), Mike Knoop (Knoop@zapier.com) and Going to Tacoma airport, Start time ⏰ is After 7:00 PM, Location is Seattle Tacoma International Airport\


Example 2
Input: The given date is 2023-11-03. Which schedule do I have on the given day?\
Output: Schedule is Watching soccer game, Start time ⏰ is After 1:00 AM

Calendar input: {event_list_new}\
\n\nInput: The given date is {date_min[:10]}. {user_input}
\n\nOutput: \
'''

print(input_text)

calendar_chatgpt.messages.append({
    "role": "user",
    "content": input_text,
})

# Call ChatGPT
response = calendar_chatgpt.call()
message = response.choices[0].message.content
print("response:", response)
print("message:", message)

# Save the returned message
# message = response["choices"][0]["message"]
# self.messages.append(message)



 You are a sophisticated calendar management assistant, adapt at organizing and managing calendar schedules for both simple and complex tasks.
For a given day, check the user's Calendar input which is given as list of python dictionaries and output the agenda for the day in markdown using relevant emojis as bullet points. Your output must be in this format. Json("date": <YYYY/MM/DD>, "schedule": <schedule>, "start_time":<HH:MM>, "Location": <location>, "Participants":<participants>)
Here's an example:
Example 1
Input: The given date is 2023-11-21. Which schedule do I have on the given day?Output: Schedule is Check-in at HyattRegency Seattle, Start time ⏰ is After 4:00 PM, Location is Hyatt Regency, Seattle, Participants areSheryl Soo(sheryl@zapier.com), Mike Knoop (Knoop@zapier.com) and Going to Tacoma airport, Start time ⏰ is After 7:00 PM, Location is Seattle Tacoma International Airport

Example 2
Input: The given date is 2023-11-03. Which schedule do I have on the given day?Output:

In [127]:
# Post-processing JSON file
from ast import literal_eval
message_dict = message.replace(' ', '')
message_dict = message_dict.replace("true", "'true'")
message_dict = eval(message_dict)

print("You have total {0} schedules for {1}, {2}.".format(len(message_dict['schedule']), date_expression, date_min[:10]))
for i in range(len(message_dict['schedule'])):
    print("schedule {0} is {1}. Start time ⏰ is {2}. ".format(i, message_dict['schedule'][i]['summary'], message_dict['schedule'][i]['start_time']), end="")
    if message_dict['schedule'][i]['Location'] != '':
        print("Location is {0}. ".format(message_dict['schedule'][i]['Location']), end="")
    if message_dict['schedule'][i]['Participants'] != '' :
        print("Participants are ", end="")
        for p in range(len(message_dict['schedule'][i]['Participants'])):
            if p == len(message_dict['schedule'][i]['Participants'])-1:
                print("and {0}. ".format(message_dict['schedule'][i]['Participants'][p]['email']), end="")
            else:
                print("{0}, ".format(message_dict['schedule'][i]['Participants'][p]['email']), end="")
            
    print('\n')
    


You have total 5 schedules for Monday two weeks later, 2023/12/04.
schedule 0 is 기숙사입주신청. Start time ⏰ is 12:15. 

schedule 1 is 종현-재윤. Start time ⏰ is 14:10. Location is https://snu-ac-kr.zoom.us/j/95468857508?pwd=T2w4dERnb3k4NUtpWi9ZRS9FY1lkUT09. Participants are hyeongoon11@snu.ac.kr, and lee.jayyoon@snu.ac.kr. 

schedule 2 is 종현-재윤. Start time ⏰ is 14:10. Location is https://snu-ac-kr.zoom.us/j/95468857508?pwd=T2w4dERnb3k4NUtpWi9ZRS9FY1lkUT09. Participants are hyeongoon11@snu.ac.kr, and lee.jayyoon@snu.ac.kr. 

schedule 3 is 데사프. Start time ⏰ is 15:30. 

schedule 4 is [P4DS]SSSCIMeeting. Start time ⏰ is 19:00. Location is https://snu-ac-kr.zoom.us/j/91376247802. Participants are anmh9161@snu.ac.kr, hyeongoon11@snu.ac.kr, hyeryung.son@snu.ac.kr, miskemad@snu.ac.kr, and yellowbill@snu.ac.kr. 

